# Harvester GetValueRequest
This is a KWH HMI which allows to request the harvester's data. This Notebook acts as an **HMI** in terms of the S³I and a **client** in terms of OAuth authentication. Enter the id of your HMI with the corresponding secret as *hmi* in this script to make this notebook to your HMI. Running this notebook, you will authorize your HMI to call up the harvester's data. 
Just go to the **Cell** drop-down menu and use the **Run All** button.

First, all necessary modules are imported into the script, including the S³I library.

In [ ]:
import s3i
import json
import uuid
import jwt
import time
import os
import base64
import getpass
import requests
import collections
from IPython.display import Image
from tools import print_with_timestamp, check_message_encryption, yes, no

## Configure the notebook
In the first step, OAuth2 specified client ID and secret for the notebook are imported, so that the notebook can be assigned as a OAuth2-client and reached via the S3I. 

In [ ]:
hmiId = "s3i:01437be2-7cc6-4cbd-97b5-684402c5a177"
hmiSecret = "d271b34b-f747-40de-a2f8-0a8637a2114b"

Next, you have to enter your username and password. With your access data a token is requested which authorizes this client (your HMI) to call up the harvester's data on your behalf.

In [ ]:
print_with_timestamp("DEMO harvester, please log in!")
username = input('[S3I]: Please enter your username:').strip('," ')
password = getpass.getpass('[S3I]: Please enter the password:')
print_with_timestamp("Your credentials are sent to S3I IdentityProvider.")
s3i_identity_provider = s3i.IdentityProvider(grant_type='password', 
                                             identity_provider_url="https://idp.s3i.vswf.dev/",
                                             realm='KWH',
                                             client_id=hmiId,
                                             client_secret=hmiSecret,
                                             username=username,
                                             password=password)
access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN)

''' decode the access token
'''
parsed_username = jwt.decode(access_token, verify=False)[
    "preferred_username"]

print_with_timestamp("Token received, " + parsed_username + " logged in.")

After the token has been received, it can be used to query the endpoint of the harvester. To query the harvester in the S³I Directory, you must have the rights to view the harvester. With the creation of your S³I account you should have got this right. In case of problems please contact the S³I team (s3i@kwh40.de).

Besides the endpoint of the harvester, the endpoint of this HMI is also needed to add as "ReplyToEndpoint" field in the request. The endpoint of this HMI is also queried from the directory, as it may have changed. By querying its own endpoint from the directory, the HMI is independent of changes made to its own endpoint. These changes then only need to be listed in the directory.

In [ ]:
''' authentication with JWT in S3I Directory 
'''

s3i_directory = s3i.Directory(
    s3i_dir_url="https://dir.s3i.vswf.dev/api/2/", token=access_token)

print_with_timestamp("Authentication with Token in S3I Directory successful")
    
''' query the sender's endpoint
'''
sender_endpoints = s3i_directory.queryThingIDBased(hmiId+"/attributes/allEndpoints")
for item in sender_endpoints:
    if "s3i:" in item:
        sender_endpoint = item
    else:
        continue

## MessageId Storage
The messageIds storage stores all message ids from the messages which have been sent. If a message arrives it is checked against these ids to see, if it is a response to one of these messages. If it is not a response to one a request send from this notebook, the message won't be parsed.

In [ ]:
messageIds = list()

# Get Harvester S³I Directory Entry

In [ ]:
%%html
<img src="harvester_images/jpy_dir.png", width=300, height=300>

In [ ]:
''' query the harvesters's Id in S3I-Directory
'''
name = input('[S3I]: Please enter the name of harvester: \n[mmi_fog_harvester]\n')
harvesterID = s3i_directory.queryAttributeBased(
    "name", name)[0]["thingId"]

''' query the entire harvesters's entry in S3I-Directory
'''
harvesterDir = s3i_directory.queryThingIDBased(harvesterID)
harvesterDir = json.loads(json.dumps(harvesterDir), object_pairs_hook=collections.OrderedDict)
print_with_timestamp(json.dumps(harvesterDir, indent=2))

''' query the harvesters's endpoints in S3I-Directory
'''
harvesterEndpoint = s3i_directory.queryThingIDBased(harvesterID+"/attributes/allEndpoints")
receivers = [harvesterID]
receiver_endpoints = harvesterEndpoint

# >>> *Run All below* from here to access the harvester via S3I-B GetValue messages
## Prepared the request
After querying the endpoints, the message is prepared. A *GetValueRequest* is instantiated and filled in.

In [ ]:
%%html
<img src="harvester_images/jpy_broker_har.png", width=700, height=700>

In [ ]:
print_with_timestamp("Prepare the GetValueRequest.")
attributePath = input(
        '[S3I]: please enter the attribute path of the value you want to receive from the harvester. Examples: \n<empty path, just press enter>  \nattributes \nfeatures \nattributes/features/ml40::Composite/targets/ml40::Engine/features/ml40::RotationalSpeed/rpm \nattributes/features/ml40::Location/longitude \nattributes/features/ml40::Location/latitude\n').strip('," ')

msg_uuid = "s3i:" + str(uuid.uuid4())

servReq = s3i.GetValueRequest()
servReq.fillGetValueRequest(senderUUID=hmiId, receiverUUIDs=receivers, sender_endpoint=sender_endpoint,
                                attributePath=attributePath, msgUUID=msg_uuid)
print_with_timestamp("GetValueRequest prepared:")
print(json.dumps(servReq.msg, indent=2))

## Send the request
The HMI requests a new token to establish a connection to the broker and sends the request to the harvester's endpoint. It then checks incoming responses.

In [ ]:
print_with_timestamp("Sending the GetValueRequest to the harvester")
access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN)
headers = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + access_token}
messageIds.append(servReq.msg["identifier"])
for item in receiver_endpoints:
    if "s3ib" in item and "https" not in item:
        s3ib_endpoint = item
    else:
        continue 
        
''' send S3I-B message to S3I-Broker API
'''
response = requests.post(url="https://broker.s3i.vswf.dev/"+s3ib_endpoint,
                                data=json.dumps(servReq.msg), headers=headers)
print_with_timestamp(response.text)

## Receive the harvester's response

In [ ]:
def receive():
    print_with_timestamp("Checking the harvester's response")
    access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN)
    headers = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + access_token}
    response = requests.get(url="https://broker.s3i.vswf.dev/"+sender_endpoint , headers=headers)
    msg_json = ""
    value_json = ""
    json_acceptable_string = response.text.replace("'", "\"")
    if json_acceptable_string:
        if check_message_encryption(response.text.strip('"')) == "pgp":
            print_with_timestamp("You received a PGP message but this notebook can not decrypt PGP messages. Use the 03_inbox notebook to receive PGP messages.")
            print_with_timestamp("PGP Message: " + response.text)
            repeat()
        else: 
            msg_json = json.loads(json_acceptable_string)
            if msg_json["replyingToMessage"] in messageIds:
                value_json = msg_json["value"]
                messageIds.remove(msg_json["replyingToMessage"])
    else:
        print_with_timestamp("The harvester did not respond yet.")
        repeat()
    return msg_json, value_json

def repeat():
    decision = input("[S3I] Do you want to check for new messages again? [j/n]")
    if decision in yes:
        receive()
    elif decision in no:
        print_with_timestamp("You do not want to check for more messages. If you want to check for new messages, just execute this cell again (Run button or SHIFT+RETURN)")
    else:
        print_with_timestamp("I could not understand your response. If you want to check for new messages, just execute this cell again (Run button or SHIFT+RETURN)")

In [ ]:
msg, value = receive()

In [ ]:
if msg:
    msg = json.loads(json.dumps(msg), object_pairs_hook=collections.OrderedDict)
    print_with_timestamp("You received a message: " + json.dumps(msg, indent=2))
    

In [ ]:
if value:
    value = json.loads(json.dumps(value), object_pairs_hook=collections.OrderedDict)
    print_with_timestamp("Your requested value is: " + json.dumps(value, indent=2))